# Uploading Batch Input File
 - batch file 보낼 때 너무 많이 보내지 말고 구분해서 보내기

In [2]:
from getpass import getpass

openai_api_key = getpass("OPENAI_API_KEY")

In [3]:
from openai import OpenAI
import json
import pandas as pd

client = OpenAI(api_key=openai_api_key)

In [39]:
batch_input_file = client.files.create(
    file=open("ExampleSentence_Voca_Elem_batch_5.jsonl","rb"),
    purpose="batch"
)

print(batch_input_file)

FileObject(id='file-CaCckc8yisYbL4A27i1nYC', bytes=790100, created_at=1740363082, filename='ExampleSentence_Voca_Elem_batch_5.jsonl', object='file', purpose='batch', status='processed', status_details=None, expires_at=None)


# Create the Batch

In [40]:
# 업로드한 배치 입력 파일 ID 저장
batch_input_file_id = batch_input_file.id
print(batch_input_file_id)

file-CaCckc8yisYbL4A27i1nYC


In [41]:
batch_info = client.batches.create(
    input_file_id = batch_input_file_id,
    endpoint = "/v1/chat/completions", # 챗봇 api
    completion_window = "24h", # 현재는 이 기능만 가능
    metadata = {
        "description": "Elem example sentence 5"
    }
)

In [42]:
response_json = json.loads(batch_info.json())
response_output = json.dumps(response_json, ensure_ascii=False, indent = 2)
print(response_output)

{
  "id": "batch_67bbd54f77188190975f31a96da7208b",
  "completion_window": "24h",
  "created_at": 1740363087,
  "endpoint": "/v1/chat/completions",
  "input_file_id": "file-CaCckc8yisYbL4A27i1nYC",
  "object": "batch",
  "status": "validating",
  "cancelled_at": null,
  "cancelling_at": null,
  "completed_at": null,
  "error_file_id": null,
  "errors": null,
  "expired_at": null,
  "expires_at": 1740449487,
  "failed_at": null,
  "finalizing_at": null,
  "in_progress_at": null,
  "metadata": {
    "description": "Elem example sentence 5"
  },
  "output_file_id": null,
  "request_counts": {
    "completed": 0,
    "failed": 0,
    "total": 0
  }
}


# Checking the Status of Batch
 - validating : the input file is being validated before the batch can begin
 - failed : the input file has failed the validation process
 - in_progress : the input file was successfully validated and the batch is currently being run
 - finalizing : the batch has completed and the results are being prepared
 - completed : the batch has been completed and the results are ready
 - expired : the batch was not able to be completed within the 24-hour time window
 - cancelling : the batch is being cancelled (may take up to 10 minutes)
 - cancelled : the batch was cancelled

*split data*
- batch_id = 'batch_67bbd4eeab748190a1ec56647bd62d34' : split1_ExampleSentence_Vcoa_Elem : 
- batch_id = 'batch_67bbd50f3ad481909c4e8d90c4af1555' : split2_ExampleSentence_Vcoa_Elem : 
- batch_id = 'batch_67bbd52465388190ad20efe15e9f59f0' : split3_ExampleSentence_Vcoa_Elem : 
- batch_id = 'batch_67bbd539e1108190aaaef685e49578f0' : split4_ExampleSentence_Vcoa_Elem : 
- batch_id = 'batch_67bbd54f77188190975f31a96da7208b' : split5_ExampleSentence_Vcoa_Elem : 

In [50]:
batch_id = "batch_67bbd4eeab748190a1ec56647bd62d34"
batch = client.batches.retrieve(batch_id)
batch_json = json.loads(batch.json())
batch_json

{'id': 'batch_67bbd4eeab748190a1ec56647bd62d34',
 'completion_window': '24h',
 'created_at': 1740362990,
 'endpoint': '/v1/chat/completions',
 'input_file_id': 'file-LX9dYGfoaDzzRYvGNj3fMy',
 'object': 'batch',
 'status': 'in_progress',
 'cancelled_at': None,
 'cancelling_at': None,
 'completed_at': None,
 'error_file_id': None,
 'errors': None,
 'expired_at': None,
 'expires_at': 1740449390,
 'failed_at': None,
 'finalizing_at': None,
 'in_progress_at': 1740362991,
 'metadata': {'description': 'Elem example sentence 1'},
 'output_file_id': None,
 'request_counts': {'completed': 467, 'failed': 0, 'total': 500}}

## 시간 확인

In [13]:
from datetime import datetime
import pytz

kst = pytz.timezone('Asia/Seoul')

# 변환 함수
def convert_timestamp(starttime, endtime):
    st = datetime.utcfromtimestamp(starttime)
    end = datetime.utcfromtimestamp(endtime)
    dt_kst_st = st.replace(tzinfo=pytz.utc).astimezone(kst)
    dt_kst_end = end.replace(tzinfo=pytz.utc).astimezone(kst)

    return f"시작 시간 : {dt_kst_st}, 끝나는 시간 : {dt_kst_end}"

convert_timestamp(starttime = batch_json['created_at'], endtime = batch_json['completed_at'])


C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_2328\1843218754.py:8: DeprecationWarning: datetime.datetime.utcfromtimestamp() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.fromtimestamp(timestamp, datetime.UTC).
  st = datetime.utcfromtimestamp(starttime)
C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_2328\1843218754.py:9: DeprecationWarning: datetime.datetime.utcfromtimestamp() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.fromtimestamp(timestamp, datetime.UTC).
  end = datetime.utcfromtimestamp(endtime)


TypeError: 'NoneType' object cannot be interpreted as an integer

# Retrieving the Results
 - file-EVCbFfNykgV9HphK2h9wsk

In [14]:
file_response = client.files.content('file-E8DfJtKogGhXc6jJqGpze6') # output_file_id 이용
print(file_response.text)

{"id": "batch_req_67b98d71475881909b05dc38910d12d3", "custom_id": "request-1", "response": {"status_code": 200, "request_id": "53751e77244f3569203d7e86deff5a22", "body": {"id": "chatcmpl-B3Efmb2j5CxtVtM9LmfSK0I7lTKir", "object": "chat.completion", "created": 1740111086, "model": "o3-mini-2025-01-31", "choices": [{"index": 0, "message": {"role": "assistant", "content": "{\n  \"example_sentence\": \"We enter the game.\",\n  \"example_sentence_kor\": \"\uc6b0\ub9ac\ub294 \uac8c\uc784\uc5d0 \ucc38\uac00\ud574\uc694.\"\n}", "refusal": null}, "finish_reason": "stop"}], "usage": {"prompt_tokens": 341, "completion_tokens": 1127, "total_tokens": 1468, "prompt_tokens_details": {"cached_tokens": 0, "audio_tokens": 0}, "completion_tokens_details": {"reasoning_tokens": 1088, "audio_tokens": 0, "accepted_prediction_tokens": 0, "rejected_prediction_tokens": 0}}, "service_tier": "default", "system_fingerprint": "fp_ef58bd3122"}}, "error": null}
{"id": "batch_req_67b98d7177888190ab782f98f7f38cb5", "cus

In [15]:
list = file_response.text.split("\n")
list = [item for item in list if item.strip()]
list
len(list)


439

In [7]:
import json

sample_output = []

# sample_list가 JSON 문자열 리스트라고 가정
for sample in list:
    sample_json = json.loads(sample)  # 문자열을 JSON 딕셔너리로 변환
    content = sample_json["response"]["body"]["choices"][0]["message"]["content"]
    content_json = json.loads(content)  # 내부 JSON 문자열도 변환

    sample_output.append(f"{sample_json["custom_id"]} 번째")
    sample_output.append(content_json)

    print(json.dumps(content_json, indent=2, ensure_ascii=False))  # 한글 출력 가능
    break


{
  "example_sentence": "I see a dog.",
  "example_sentence_kor": "나는 하나의 개를 봐요."
}


# batch 결과

In [8]:
for sample in list:
    sample_json = json.loads(sample)
    content = sample_json["response"]["body"]["choices"][0]["message"]["content"]
    content_json = json.loads(content)
    content_json['example_sentence'] = content_json['example_sentence'].replace("\t"," ")
    content_json['example_sentence_kor'] = content_json['example_sentence_kor'].replace("\t"," ")
    print(json.dumps(content_json, indent=2, ensure_ascii=False))
    break

{
  "example_sentence": "I see a dog.",
  "example_sentence_kor": "나는 하나의 개를 봐요."
}


# CSV 파일로 변환

In [9]:
def JsonToCSV(data:json, file_name:str) -> json:
    data_list = []

    for sample in data:
        # json 변환환
        sample_json = json.loads(sample)
        content = sample_json["response"]["body"]["choices"][0]["message"]["content"]
        content_json = json.loads(content)
        content_json['example_sentence'] = content_json['example_sentence'].replace("\t"," ")
        content_json['example_sentence_kor'] = content_json['example_sentence_kor'].replace("\t"," ")

        # 학습 목표
        example_sentence = content_json["example_sentence"]
        example_sentence_kor = content_json["example_sentence_kor"]

        # 데이터 만들기
        csv_data = {
            "example_english": example_sentence,
            "example_korean": example_sentence_kor,
        }

        data_list.append(csv_data)

    df = pd.DataFrame(data_list)

    # CSV 저장
    df.to_csv(f"{file_name}.csv", index=False, encoding='utf-8-sig')

    print(f"CSV 변환 완료 파일명: {file_name}.csv")


In [10]:
name = 'ExampleSentence_voca_batch_1'
JsonToCSV(list, name)

CSV 변환 완료 파일명: ExampleSentence_voca_batch_1.csv


In [11]:
len(list)

499

In [273]:
df1 = pd.read_csv('ExampleSentence_voca_batch_1.csv')
df2 = pd.read_csv('ExampleSentence_voca_batch_2.csv')
df3 = pd.read_csv('ExampleSentence_voca_batch_3.csv')
df4 = pd.read_csv('ExampleSentence_voca_batch_4.csv')
df5 = pd.read_csv('ExampleSentence_voca_batch_5.csv')
df6 = pd.read_csv('ExampleSentence_voca_batch_6.csv')
df7 = pd.read_csv('ExampleSentence_voca_batch_7.csv')

merged_data = pd.concat([df1, df2, df3, df4, df5, df6, df7], axis=0, ignore_index=True)
merged_data.shape
merged_data.to_csv("ExampleSentence_Voca_Mid(total).csv", encoding="utf-8-sig")

# Cancelling a Batch

In [237]:
# cancel_batch = client.batches.cancel("")
# batch_json = json.loads(cancel_batch.json())
# batch_ouput = json.dumps(batch_json, ensure_ascii=False, indent=2)
# print(batch_ouput)